In [75]:
!pip install geemap
import geopandas as gpd, pandas as pd, os, numpy as np
import ee, geemap
import matplotlib.pyplot as plt
import pprint

You should consider upgrading via the 'C:\Users\Goldferret\PycharmProjects\NDMN-Jupyter-Notebook\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [76]:
gdf=gpd.read_file('Clasification_Plots.zip')

# Displays the unique values of the 'Uso' column
display(gdf['Uso'].value_counts(dropna=False))
display(gdf['Uso'].value_counts(dropna=False).sum())

Uso
Bosque                 58708
Pastos                 23291
Agricultura             8398
Otras clases            5449
Humedal                 3926
Plantacion forestal      902
Sin informacion          270
None                     216
Name: count, dtype: int64

np.int64(101160)

In [94]:
# Creates subsets for every type of 'Uso'
subset_bosque = gdf[gdf['Uso'].isin(['Bosque'])]
subset_pastos = gdf[gdf['Uso'].isin(['Pastos'])]
subset_agricultura = gdf[gdf['Uso'].isin(['Agricultura'])]
subset_otras = gdf[gdf['Uso'].isin(['Otras clases'])]
subset_humedal = gdf[gdf['Uso'].isin(['Humedal'])]
subset_plantacion = gdf[gdf['Uso'].isin(['Plantacion forestal'])]
subset_sin = gdf[gdf['Uso'].isin(['Sin informacion'])]
subset_none = gdf[gdf['Uso'].isna()]

# Displays the unique values of the Sin informacion subset
print("Unique values of Sin informacion plots' Cobertura:")
display(subset_none['Cobertura'].value_counts(dropna=False))
display(subset_none['Cobertura'].value_counts(dropna=False).sum())

Unique values of Sin informacion plots' Cobertura:


Cobertura
None    216
Name: count, dtype: int64

np.int64(216)

In [90]:
# Specify the columns for which you want to find unique values
columns_of_interest = ['Cobertura', 'Vegetacion', 'Herbaceas', 'Pasto_Arb', 'Cultivo', 'Humedal', 'Terreno', 'Agua', 'Otra_clase', 'SAF', 'Cambios15_', 'Gana_Perdi']

# Use apply with unique
unique_values = {col: subset_none[col].unique().tolist() for col in columns_of_interest}

# Print out the dictionary with pretty print
# This will show which columns "should not" apply to when the plot is 'Bosque' (i.e. When the only unique value in the column in 'None')
pprint.pprint(unique_values, sort_dicts=False)

{'Cobertura': [None],
 'Vegetacion': [None],
 'Herbaceas': [None],
 'Pasto_Arb': [None],
 'Cultivo': [None],
 'Humedal': [None],
 'Terreno': [None],
 'Agua': [None],
 'Otra_clase': [None],
 'SAF': [None],
 'Cambios15_': [None],
 'Gana_Perdi': [None]}


In [103]:
# Used to check if there are any entries/records which are double counted

sampleid_counts = gdf['sampleid'].value_counts(dropna=False)
plotid_counts = gdf['plotid'].value_counts(dropna=False)

print("Unique values of sampleid in the original dataframe:")
display(sampleid_counts[sampleid_counts > 1])
display(sampleid_counts[sampleid_counts > 1].sum())

print("Unique values of plotid in the original dataframe:")
display(plotid_counts[plotid_counts > 9])
display(plotid_counts[plotid_counts > 9].sum())

Unique values of sampleid in the original dataframe:


sampleid
12417    4
12601    4
12657    4
12597    4
12605    4
        ..
4267     2
4266     2
11778    2
11779    2
11780    2
Name: count, Length: 41437, dtype: int64

np.int64(93128)

Unique values of plotid in the original dataframe:


plotid
2175    18
4607    18
6916    18
3150    18
3105    18
3163    18
1630    18
Name: count, dtype: int64

np.int64(126)

In [101]:
# Prints the rows which have multi-counted a sampleid
multi_sample_df = gdf[gdf['sampleid'] == 12417].index
display(multi_sample_df)
display(gdf[gdf['sampleid'] == 12417])

Index([998, 1003, 1008, 1009], dtype='int64')

,Source.Nam,plotid,sampleid,lon,lat,sample_geo,Uso,Cobertura,Vegetacion,Herbaceas,Pasto_Arb,Cultivo,Humedal,Terreno,Agua,Otra_clase,SAF,Cambios15_,Gana_Perdi,geometry
998,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,3103,12417,-84.679716,10.919648,POINT(-84.67971580866356 10.919648384406656),Agricultura,Vegetacion,Herbaceas,Gramineas,None,Otro,None,None,None,None,Cultivo mezclado (70-90%),No,None,POINT (-84.67972 10.91965)
1003,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,3104,12417,-84.661134,10.930125,POINT(-84.66113400000002 10.930125000000018),Agricultura,Vegetacion,Herbaceas,Gramineas,None,Otro,None,None,None,None,Cultivo mezclado (70-90%),No,None,POINT (-84.66113 10.93013)
1008,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,3105,12417,-84.661517,10.907986,POINT(-84.66151719133643 10.907985599578808),Pastos,Vegetacion,Herbaceas,Gramineas,Pastos mezclados (70-90%),None,None,None,None,None,None,No,None,POINT (-84.66152 10.90799)
1009,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,3105,12417,-84.661517,10.907986,POINT(-84.66151719133643 10.907985599578808),Plantacion forestal,Vegetacion,Arboles,None,None,None,None,None,None,None,None,No,None,POINT (-84.66152 10.90799)
